# Scraping and Populating Data Frame

In [149]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import colors
from IPython.core.display import HTML

In [150]:
url = 'http://pokemondb.net/pokedex/all'
req = requests.get(url)
page = req.text
soup = BeautifulSoup(page, 'html.parser')

In [151]:
HTML(str(soup.table))

#,Name,Type,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed
001,Bulbasaur,GrassPoison,318,45,49,49,65,65,45
002,Ivysaur,GrassPoison,405,60,62,63,80,80,60
003,Venusaur,GrassPoison,525,80,82,83,100,100,80
003,VenusaurMega Venusaur,GrassPoison,625,80,100,123,122,120,80
004,Charmander,Fire,309,39,52,43,60,50,65
005,Charmeleon,Fire,405,58,64,58,80,65,80
006,Charizard,FireFlying,534,78,84,78,109,85,100
006,CharizardMega Charizard X,FireDragon,634,78,130,111,130,85,100
006,CharizardMega Charizard Y,FireFlying,634,78,104,78,159,115,100
007,Squirtle,Water,314,44,48,65,50,64,43


In [152]:
rows = [row for row in soup.find('table').find_all('tr')]
rows[800]

<tr>\n<td class="num cell-icon-string" data-sort-value="721"><i class="pki" data-sprite="pkiAll n721"></i> 721</td> <td class="cell-icon-string"><a class="ent-name" href="/pokedex/volcanion" title="View pokedex for #721 Volcanion">Volcanion</a></td><td class="cell-icon"><a class="type-icon type-fire" href="/type/fire">Fire</a><br><a class="type-icon type-water" href="/type/water">Water</a></br></td>\n<td class="num-total">600</td>\n<td class="num">80</td>\n<td class="num">110</td>\n<td class="num">120</td>\n<td class="num">130</td>\n<td class="num">90</td>\n<td class="num">70</td>\n</tr>

In [153]:
columns = [col.get_text() for col in rows[0].find_all('th') if col.get_text() != '']
columns

[u'#',
 u'Name',
 u'Type',
 u'Total',
 u'HP',
 u'Attack',
 u'Defense',
 u'Sp. Atk',
 u'Sp. Def',
 u'Speed']

In [154]:
indexes = [i for i in range(0, len(rows)-1)]
indexes

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


In [155]:
values = [value.get_text() for row in rows[1:] for value in row.find_all('td')] 
values

[u' 001',
 u'Bulbasaur',
 u'GrassPoison',
 u'318',
 u'45',
 u'49',
 u'49',
 u'65',
 u'65',
 u'45',
 u' 002',
 u'Ivysaur',
 u'GrassPoison',
 u'405',
 u'60',
 u'62',
 u'63',
 u'80',
 u'80',
 u'60',
 u' 003',
 u'Venusaur',
 u'GrassPoison',
 u'525',
 u'80',
 u'82',
 u'83',
 u'100',
 u'100',
 u'80',
 u' 003',
 u'VenusaurMega Venusaur',
 u'GrassPoison',
 u'625',
 u'80',
 u'100',
 u'123',
 u'122',
 u'120',
 u'80',
 u' 004',
 u'Charmander',
 u'Fire',
 u'309',
 u'39',
 u'52',
 u'43',
 u'60',
 u'50',
 u'65',
 u' 005',
 u'Charmeleon',
 u'Fire',
 u'405',
 u'58',
 u'64',
 u'58',
 u'80',
 u'65',
 u'80',
 u' 006',
 u'Charizard',
 u'FireFlying',
 u'534',
 u'78',
 u'84',
 u'78',
 u'109',
 u'85',
 u'100',
 u' 006',
 u'CharizardMega Charizard X',
 u'FireDragon',
 u'634',
 u'78',
 u'130',
 u'111',
 u'130',
 u'85',
 u'100',
 u' 006',
 u'CharizardMega Charizard Y',
 u'FireFlying',
 u'634',
 u'78',
 u'104',
 u'78',
 u'159',
 u'115',
 u'100',
 u' 007',
 u'Squirtle',
 u'Water',
 u'314',
 u'44',
 u'48',
 u'65',

In [156]:
stacked_values = zip(*[values[i::10] for i in range(len(columns))])
stacked_values

[(u' 001',
  u'Bulbasaur',
  u'GrassPoison',
  u'318',
  u'45',
  u'49',
  u'49',
  u'65',
  u'65',
  u'45'),
 (u' 002',
  u'Ivysaur',
  u'GrassPoison',
  u'405',
  u'60',
  u'62',
  u'63',
  u'80',
  u'80',
  u'60'),
 (u' 003',
  u'Venusaur',
  u'GrassPoison',
  u'525',
  u'80',
  u'82',
  u'83',
  u'100',
  u'100',
  u'80'),
 (u' 003',
  u'VenusaurMega Venusaur',
  u'GrassPoison',
  u'625',
  u'80',
  u'100',
  u'123',
  u'122',
  u'120',
  u'80'),
 (u' 004',
  u'Charmander',
  u'Fire',
  u'309',
  u'39',
  u'52',
  u'43',
  u'60',
  u'50',
  u'65'),
 (u' 005',
  u'Charmeleon',
  u'Fire',
  u'405',
  u'58',
  u'64',
  u'58',
  u'80',
  u'65',
  u'80'),
 (u' 006',
  u'Charizard',
  u'FireFlying',
  u'534',
  u'78',
  u'84',
  u'78',
  u'109',
  u'85',
  u'100'),
 (u' 006',
  u'CharizardMega Charizard X',
  u'FireDragon',
  u'634',
  u'78',
  u'130',
  u'111',
  u'130',
  u'85',
  u'100'),
 (u' 006',
  u'CharizardMega Charizard Y',
  u'FireFlying',
  u'634',
  u'78',
  u'104',
  u'78',

In [157]:
df = pd.DataFrame(stacked_values, columns = columns, index = indexes)
df.head()

,#,Name,Type,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed
0,001,Bulbasaur,GrassPoison,318,45,49,49,65,65,45
1,002,Ivysaur,GrassPoison,405,60,62,63,80,80,60
2,003,Venusaur,GrassPoison,525,80,82,83,100,100,80
3,003,VenusaurMega Venusaur,GrassPoison,625,80,100,123,122,120,80
4,004,Charmander,Fire,309,39,52,43,60,50,65


# Data Cleaning

Time to clean the data we just scraped. For example, as you can see in the 4th row, the Name of the pokemon is VenusaurMega Venusaur. Similarly, in all the rows which have dual type pokemon have the type stored as Type1Type2.

We do not want that. Hence, we clean it!

In [158]:
# Replaces Word1Word2 Word3 with Word1 ( Word2 Word3 )
def clean_name(name):
    # name = 'VenusaurMega Venusaur'
    letters = [x for x in name]
    for i in range(1, len(letters)):
        if letters[i].isupper():
            letters[i] = ' ' + letters[i]
    fin_letters = ''.join(letters).split(' ')
    if len(fin_letters)>1:
        fin_letters.insert(1, '(')
        fin_letters.append(')')
    fin_name = ' '.join(fin_letters)
    return fin_name

# Replaces Type1Type2.. with Type1 Type2 ..
def clean_types(types):
    letters = [x for x in types]
    for i in range(1, len(types)):
        if letters[i].isupper():
            letters[i] = ' ' + letters[i]
    fin_type = ''.join(letters).split(' ')
    return fin_type


    

In [159]:
df['Name'] = df['Name'].apply(clean_name)
df['Type'] = df['Type'].apply(clean_types)
df.head()

# clean_name('VenusaurMega Venusaur')

,#,Name,Type,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed
0,001,Bulbasaur,"[Grass, Poison]",318,45,49,49,65,65,45
1,002,Ivysaur,"[Grass, Poison]",405,60,62,63,80,80,60
2,003,Venusaur,"[Grass, Poison]",525,80,82,83,100,100,80
3,003,Venusaur ( Mega Venusaur ),"[Grass, Poison]",625,80,100,123,122,120,80
4,004,Charmander,[Fire],309,39,52,43,60,50,65


In [160]:
df.to_json('PokemonData.json')

In [189]:
df = pd.read_json('PokemonData.json')
df=df.set_index(['#'])
df

,Attack,Defense,HP,Name,Sp. Atk,Sp. Def,Speed,Total,Type
#,,,,,,,,,
1,49,49,45,Bulbasaur,65,65,45,318,"[Grass, Poison]"
2,62,63,60,Ivysaur,80,80,60,405,"[Grass, Poison]"
8,63,80,59,Wartortle,65,80,58,405,[Water]
93,50,45,45,Haunter,115,55,95,405,"[Ghost, Poison]"
94,65,60,60,Gengar,130,75,110,500,"[Ghost, Poison]"
94,65,80,60,Gengar ( Mega Gengar ),170,95,130,600,"[Ghost, Poison]"
95,45,160,35,Onix,30,45,70,385,"[Rock, Ground]"
96,48,45,60,Drowzee,43,90,42,328,[Psychic]
97,73,70,85,Hypno,73,115,67,483,[Psychic]


# Analysis

Time to analyse the data we just scraped and structured!

In [187]:
def max_stats(df, columns):
    max_message = ''
    for col in columns:
        stat = df[col].max()
        name = df[df[col] == df[col].max()]['Name'].values[0]
        max_message += name+' has the greatest '+col+' of '+str(stat)+'.\n'
    return max_message

def min_stats(df, col_list):
    '''Get Pokemon lowest value of the column in the Data Frame'''
    message = ''
    for col in col_list:
        stat = df[col].min()
        name = df[df[col]==df[col].min()]['Name'].values[0]
        message += name+' has the worst '+col+' of '+str(stat)+'.\n'
    return message


In [188]:
stats = ['Attack', 'Defense','HP', 'Sp. Atk','Sp. Def','Speed','Total']

print max_stats(df, stats)

print min_stats(df, stats)

Mewtwo ( Mega  Mewtwo  X ) has the greatest Attack of 190.
Steelix ( Mega  Steelix ) has the greatest Defense of 230.
Blissey has the greatest HP of 255.
Mewtwo ( Mega  Mewtwo  Y ) has the greatest Sp. Atk of 194.
Shuckle has the greatest Sp. Def of 230.
Deoxys ( Speed  Forme ) has the greatest Speed of 180.
Mewtwo ( Mega  Mewtwo  X ) has the greatest Total of 780.

Chansey has the worst Attack of 5.
Chansey has the worst Defense of 5.
Shedinja has the worst HP of 1.
Shuckle has the worst Sp. Atk of 10.
Caterpie has the worst Sp. Def of 20.
Shuckle has the worst Speed of 5.
Sunkern has the worst Total of 180.

